In [13]:
# Settings

# General settings
data_folder = "/home/andrea/CIMA/PROJECT_Volta/data/integrated_data/" #"/home/andrea/Desktop/Test/series/" 
start_analysis = "2005-01-01 00:00"
end_analysis = "2020-12-31 23:00"
freq="D"

# Runoff coefficient 
basin_rainfall_fld = "/home/andrea/CIMA/PROJECT_Volta/data/rainfall_analysis/"
min_percentage = 50

In [14]:
# Imports
%matplotlib inline

from ipywidgets import interactive,HBox
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
        
import ipywidgets as widgets
from IPython.display import display, HTML
import os, glob
import datetime as dt
import matplotlib.lines as mlines

# Define custom functions
def create_df(choices):
    df = pd.DataFrame(index=pd.date_range(start_time,end_time,freq=freq), columns=choices)
    for name in choices:
        series = pd.read_csv(os.path.join(data_folder, name + ".csv"), index_col=0, header=0, parse_dates=True)
        df[name] = series.reindex(pd.date_range(start_time,end_time,freq=freq), method=None)
    return df

def multiplot(widg):
    choices = widg['new']
    df = create_df(choices)
    data = df.loc[:, choices] if choices else df
    output.clear_output(wait=True)
    with output:
        ax = data.plot(figsize=(10,7))
        plt.show()
        
def combinedplot(widg):
    choices = widg['new']
    data = series["discharge"].loc[:, choices] if choices else series
    data_rain = series["mm_h"].loc[:, choices] if choices else series
    output2.clear_output(wait=True)
    with output2:
        ax = data_rain.plot(figsize=(15,7), color='b')
        plt.ylim(bottom=0)
        plt.ylabel(("mm"))
        ax2 = ax.twinx()
        ax1 = data.plot(figsize=(15,7), color='r')
        plt.ylim(bottom=0)
        plt.ylabel(("m3/s"))
        blue_line = mlines.Line2D([], [], color='blue', label='rain')
        red_line = mlines.Line2D([], [], color='red', label='discharge')
        plt.legend(handles=[blue_line, red_line])
        plt.show()

In [15]:
# Set timing
start_time = dt.datetime.strptime(start_analysis, "%Y-%m-%d %H:%M")
end_time = dt.datetime.strptime(end_analysis, "%Y-%m-%d %H:%M")

In [16]:
# Read files and extract station names
files = glob.glob(os.path.join(data_folder,"*.csv"))
opts = sorted([os.path.basename(i).replace(".csv","") for i in files])

# Generate list
selector = widgets.SelectMultiple(
options=opts,
value=[opts[1]],
rows=len(opts),
disabled=False)

output = widgets.Output()

# Set layout and display
form_item_layout = widgets.Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)
display(widgets.HBox([selector, output], layout=form_item_layout))

# Re-generate multiplot
selector.observe(multiplot, names='value')

## Runoff coefficient analysis

In [5]:
series = {}
for data_type in ["mm_h","m3_s"]:
    series[data_type] = pd.read_csv(os.path.join(basin_rainfall_fld,"average_rainfall_" + data_type + ".txt"), index_col=0, header=0, parse_dates=True)
    series[data_type].replace([np.inf, -np.inf], 0, inplace=True)
    
if freq != "H":
    series["mm_h"] = series["mm_h"].resample(freq).agg(pd.Series.sum, skipna=False)
    series["m3_s"] = series["m3_s"].resample(freq).agg(pd.Series.mean, skipna=False)
    
missing_data = [i for i in series["mm_h"].columns if not os.path.isfile(os.path.join(data_folder, i + ".csv"))]
display("WARNING! Data for sections " + ", ".join(missing_data) + " are missing!")

series["discharge"] = create_df([i for i in series["mm_h"].columns if i not in missing_data])

'WARNING! Data for sections Bayé, Gorée, Pletou are missing!'

In [6]:
# Analyse series
series["discharge"].dropna(axis=1, how="all", inplace=True)

availability = (100*(series["discharge"].resample("Y").count()/series["m3_s"].resample("Y").count()).dropna(axis=1, how="all").dropna(axis=0, how="all")).astype("int")
runoff_coefficient = (series["discharge"].resample("Y").agg(pd.Series.sum, skipna=True)/series["m3_s"].resample("Y").agg(pd.Series.sum, skipna=True)).dropna(axis=1, how="all").dropna(axis=0, how="all")
runoff_coefficient[availability<=min_percentage] = np.nan

availability.set_index(np.array([str(i) + "_avail" for i in availability.index.year]), inplace=True)
runoff_coefficient.set_index(np.array([str(i) + "_coeff" for i in runoff_coefficient.index.year]), inplace=True)

with pd.option_context('display.float_format', '{:0.3f}'.format):
    display(pd.concat([availability.T, runoff_coefficient.T], axis=1))


,2018_avail,2019_avail,2020_avail,2018_coeff,2019_coeff,2020_coeff
Badara,79,81,85,0.067,0.037,0.063
Bagre_aval_au_pont,78,100,70,0.003,0.000,0.004
Bamboi,100,100,0,0.104,0.126,nan
Banzon,82,72,67,0.127,0.068,0.148
Batié,57,29,16,0.063,nan,nan
Bissiga,98,99,96,0.067,0.030,0.055
Bittou,74,73,71,0.208,0.007,0.152
Bongoulou,16,0,0,nan,nan,nan
Boromo,86,89,90,0.015,0.017,0.048
Bui,100,100,100,0.156,0.152,0.146


In [7]:
# Generate list
selector2 = widgets.Dropdown(
options=availability.T.index,
value=availability.T.index[0])

output2 = widgets.Output()

# Set layout and display
form_item_layout = widgets.Layout(
    display='flex',
    justify_content='space-between'
)
display(widgets.VBox([selector2, output2], layout=form_item_layout))

# Re-generate multiplot
selector2.observe(combinedplot, names='value')